In [35]:
import argparse
import numpy as np
# from mayavi import mlab 

In [36]:
def generate_cot_matrix(vertices, triangles):
    
    numv = vertices.shape[0]
    numt = triangles.shape[0]

    cot_matrix = np.zeros((numv, numv))
    scaling_factor = np.ones((numv, numv))

    for i in range(numv):
        
        req_t = triangles[(triangles[:, 0] == i) | (triangles[:, 1] == i) | (triangles[:, 2] == i)]
        
        for j in range(req_t.shape[0]):
            
                
            nbhr = [v for v in req_t[j] if v != i]

            vec1 = (vertices[nbhr[0]] - vertices[i]) / np.linalg.norm(vertices[nbhr[0]] - vertices[i], 2)
            vec2 = (vertices[nbhr[1]] - vertices[i]) / np.linalg.norm(vertices[nbhr[1]] - vertices[i], 2)
            angle_at_x = np.arccos(np.dot(vec1, vec2))

            if angle_at_x > np.pi / 2:
                scaling_factor[i, nbhr[0]] = 1 / 2



            vec1a = (vertices[i] - vertices[nbhr[0]]) / np.linalg.norm(vertices[i] - vertices[nbhr[0]], 2)
            vec2a = (vertices[nbhr[1]] - vertices[nbhr[0]]) / np.linalg.norm(vertices[nbhr[1]] - vertices[nbhr[0]], 2)

            inner_prod = np.dot(vec1a, vec2a)
            angle = np.arccos(inner_prod)

            if angle > np.pi / 2:
                scaling_factor[i, nbhr[0]] = 1 / 4

            cot_matrix[i, nbhr[1]] += 1 / np.tan(angle)

            vec1b = (vertices[i] - vertices[nbhr[1]]) / np.linalg.norm(vertices[i] - vertices[nbhr[1]], 2)
            vec2b = (vertices[nbhr[0]] - vertices[nbhr[1]]) / np.linalg.norm(vertices[nbhr[0]] - vertices[nbhr[1]], 2)

            inner_prod = np.dot(vec1b, vec2b)
            angle = np.arccos(inner_prod)

            if angle > np.pi / 2:
                scaling_factor[i, nbhr[1]] = 1 / 4

            cot_matrix[i, nbhr[0]] += 1 / np.tan(angle)
    
    return cot_matrix, scaling_factor
 

In [37]:
def get_diff_norm(vertices):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    diff_norm = np.zeros((numv, numv))
    diff_norm = np.linalg.norm(vertices.reshape(-1, 1, 3) - vertices, 2, axis=2)**2
#     for i in range(numv):
#         for j in range(numv):
#             diff_norm[i, j] = np.linalg.norm(vertices.reshape(-1, 1, 3) - vertices, 2, axis=2)**2  
    return diff_norm

In [94]:
def get_heron_area(a, b, c):
    
    x = np.linalg.norm((b - a), 2)
    y = np.linalg.norm((c - a), 2)
    z = np.linalg.norm((c - b), 2)
    
    s = (x + y + z) * 0.5
    
    return (s * (s - x) * (s - y) * (s - z)) ** 2

In [92]:
def calc_A_mixed2(vertices, triangles, cot_matrix):
    numv = vertices.shape[0]
    numt = triangles.shape[0]

    diff_norm = get_diff_norm(vertices)

    A_mixed = np.zeros((numv, numt))

    for i in range(numv):

        req_t = triangles[(triangles[:, 0] == i) | (
            triangles[:, 1] == i) | (triangles[:, 2] == i)]

        for j in range(len(req_t)):

            # tid = triangles.index(req_t[j])
            tid = np.where(np.all(triangles == req_t[j], axis=1))

            nbhr = [v for v in req_t[j] if v != i]

            vec1 = (vertices[nbhr[0]] - vertices[i]) / \
                np.linalg.norm(vertices[nbhr[0]] - vertices[i], 2)
            vec2 = (vertices[nbhr[1]] - vertices[i]) / \
                np.linalg.norm(vertices[nbhr[1]] - vertices[i], 2)
            angle_at_x = np.arccos(np.dot(vec1, vec2))

            if angle_at_x > np.pi / 2:
                A_mixed[i, tid] = get_heron_area(
                    vertices[i], vertices[nbhr[0]], vertices[nbhr[1]]) / 2
                continue

            vec1a = (vertices[i] - vertices[nbhr[0]]) / \
                np.linalg.norm(vertices[i] - vertices[nbhr[0]], 2)
            vec2a = (vertices[nbhr[1]] - vertices[nbhr[0]]) / \
                np.linalg.norm(vertices[nbhr[1]] - vertices[nbhr[0]], 2)

            inner_prod = np.dot(vec1a, vec2a)
            angle = np.arccos(inner_prod)

            if angle > np.pi / 2:
                A_mixed[i, tid] = get_heron_area(
                    vertices[i], vertices[nbhr[0]], vertices[nbhr[1]]) / 4
                continue

#             tan1 = np.tan(angle)

            vec1b = (vertices[i] - vertices[nbhr[1]]) / \
                np.linalg.norm(vertices[i] - vertices[nbhr[1]], 2)
            vec2b = (vertices[nbhr[0]] - vertices[nbhr[1]]) / \
                np.linalg.norm(vertices[nbhr[0]] - vertices[nbhr[1]], 2)

            inner_prod = np.dot(vec1b, vec2b)
            angle = np.arccos(inner_prod)

            if angle > np.pi / 2:
                A_mixed[i, tid] = get_heron_area(
                    vertices[i], vertices[nbhr[0]], vertices[nbhr[1]]) / 4
                continue

#             tan2 = np.tan(angle)

            A_v_of_tid = 0.125 * (cot_matrix[i, nbhr[0]] * diff_norm[
                                  i, nbhr[0]] + cot_matrix[i, nbhr[1]] * diff_norm[i, nbhr[1]])
            A_mixed[i, tid] = A_v_of_tid

    return np.sum(A_mixed, axis=1)

            
        

In [101]:
def calc_A_mixed3(vertices, triangles):
    
    numv = vertices.shape[0]
    numt = triangles.shape[0]

    diff_norm = get_diff_norm(vertices)

    A_mixed = np.zeros((numv, numt))
    
    for i in range(numv):

        req_t = triangles[(triangles[:, 0] == i) | (
            triangles[:, 1] == i) | (triangles[:, 2] == i)]
        
        for j in range(len(req_t)):
        
            tid = np.where(np.all(triangles == req_t[j], axis=1))

            nbhr = [v for v in req_t[j] if v != i]

            vec1 = (vertices[nbhr[0]] - vertices[i]) / \
                np.linalg.norm(vertices[nbhr[0]] - vertices[i], 2)
            vec2 = (vertices[nbhr[1]] - vertices[i]) / \
                np.linalg.norm(vertices[nbhr[1]] - vertices[i], 2)
            angle_at_x = np.arccos(np.dot(vec1, vec2))

            if angle_at_x > np.pi / 2:
                A_mixed[i, tid] = get_heron_area(
                    vertices[i], vertices[nbhr[0]], vertices[nbhr[1]]) / 2
                continue

            vec1a = (vertices[i] - vertices[nbhr[0]]) / \
                np.linalg.norm(vertices[i] - vertices[nbhr[0]], 2)
            vec2a = (vertices[nbhr[1]] - vertices[nbhr[0]]) / \
                np.linalg.norm(vertices[nbhr[1]] - vertices[nbhr[0]], 2)

            inner_prod = np.dot(vec1a, vec2a)
            angle1 = np.arccos(inner_prod)

            if angle1 > np.pi / 2:
                A_mixed[i, tid] = get_heron_area(
                    vertices[i], vertices[nbhr[0]], vertices[nbhr[1]]) / 4
                continue

    #             tan1 = np.tan(angle)

            vec1b = (vertices[i] - vertices[nbhr[1]]) / \
                np.linalg.norm(vertices[i] - vertices[nbhr[1]], 2)
            vec2b = (vertices[nbhr[0]] - vertices[nbhr[1]]) / \
                np.linalg.norm(vertices[nbhr[0]] - vertices[nbhr[1]], 2)

            inner_prod = np.dot(vec1b, vec2b)
            angle2 = np.arccos(inner_prod)

            if angle2 > np.pi / 2:
                A_mixed[i, tid] = get_heron_area(
                    vertices[i], vertices[nbhr[0]], vertices[nbhr[1]]) / 4
                continue

            A_v_of_tid = 0.125 * (((1 / np.tan(angle1)) * np.linalg.norm(vertices[i] - nbhr[1], 2)) + ((1 / np.tan(angle2)) * np.linalg.norm(vertices[i] - nbhr[0], 2))) 
            A_mixed[i, tid] = A_v_of_tid
        
    return np.sum(A_mixed, axis=1)

    

In [38]:
def calc_A_mixed(vertices, cot_matrix, scaling_factor):
#     cot_matrix, scaling_factor = generate_cot_matrix(vertices, triangles)
    diff_norm = get_diff_norm(vertices)
    A_mixed = np.sum(cot_matrix * diff_norm * scaling_factor, axis = 1) / 8
    return A_mixed

In [39]:
def get_diff(vertices):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    diff_x = np.zeros((numv, numv, 3))
    diff_x = vertices.reshape(-1, 1, 3) - vertices
#     for i in range(numv):
#         for j in range(numv):
#             diff_x[i, j] = vertices[i] - vertices[j]
    return diff_x

In [40]:
def mean_curvature_normal_operator(vertices, triangles, A_mixed, cot_matrix):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    diff_x = get_diff(vertices)
    K = np.zeros((numv, numv, 3))
    for i in range(numv):
        K[i, :] = (cot_matrix[i, :] * diff_x[i, :].T).T / (2 * A_mixed[i])
    K = np.sum(K, axis=1)
    return K

In [41]:
def get_mean_curvature(vertices, triangles, A_mixed, cot_matrix):
    K = mean_curvature_normal_operator(vertices, triangles, A_mixed, cot_matrix)
    K_H = 0.5 * np.linalg.norm(K, 2, axis=1)
    return K_H

In [42]:
def get_gaussian_curvature(vertices, triangles, A_mixed):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    K_G = np.zeros(numv)
    for i in range(numv):
        sum_theta = 0
        req_t = triangles[(triangles[:, 0] == i) | (triangles[:, 1] == i) | (triangles[:, 2] == i)]

        for j in range(req_t.shape[0]):

            nbhrs = [v for v in req_t[j] if v != i]
            vec1 = vertices[nbhrs[0]] - vertices[i]
            vec1 = vec1 / np.linalg.norm(vec1, 2)
            vec2 = vertices[nbhrs[1]] - vertices[i]
            vec2 = vec2 / np.linalg.norm(vec2, 2)

            angle = np.arccos(np.dot(vec1, vec2))
            sum_theta+=angle
        K_G[i] = ((2 * np.pi) - sum_theta) / A_mixed[i]
    return K_G

In [43]:
def get_principal_curvatures(K_H, K_G):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    zeros = np.zeros(numv)
    delx = np.sqrt(np.max(np.vstack((K_H**2 - K_G, zeros)), axis=0))
    K_1 = K_H + delx
    K_2 = K_H - delx
    return K_1, K_2

In [44]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return np.array(verts, dtype=np.float64), np.array(faces, dtype=np.int64)

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--calc", help="Flag for calculation mode", action="store_true")
    parser.add_argument("i", help="Path to input file", type=str)
    parser.add_argument("--op", help="Sets the operation\n1 - Mean Curvature\n2 - Gaussian Curvature\n3 - Principal Curvatures", type=int)
    parser.add_argument("-o", help="Path to output file", type=str, default=None)
    parser.add_argument("--mesh", help="Required for plotting the figure", default=None)
    args = parser.parse_args()
    if args.calc == True:
        op = args.op
        if op is None:
            parser.error("--calc requires --op")
        mesh_file = args.i
        f = open(mesh_file)
        vertices, triangles = read_off(f)
        cot_matrix, scaling_factor = generate_cot_matrix(vertices, triangles)
        A_mixed = calc_A_mixed(vertices, cot_matrix, scaling_factor)
        
        K_H = None
        K_G = None
        K_1 = None
        K_2 = None
        
        if op == 1:
            K_H = get_mean_curvature(vertices, triangles, A_mixed, cot_matrix)
            if args.o is None:
                np.save(args.i.split(".")[0] + "_KH.npy", K_H)
        if op == 2:
            K_G = get_gaussian_curvature(vertices, triangles, A_mixed)
            if args.o is None:
                np.save(args.i.split(".")[0] + "_KN.npy", K_G)
        if op == 3:
            K_H = get_mean_curvature(vertices, triangles, A_mixed, cot_matrix)
            K_G = get_gaussian_curvature(vertices, triangles, A_mixed)
            K_1, K_2 = get_principal_curvatures(K_H, K_G)
            if args.o is None:
                np.save(args.i.split(".")[0] + "_K1.npy", K_1)
                np.save(args.i.split(".")[0] + "_K2.npy", K_2)
    else:
        from mayavi import mlab 
        f = np.load(args.i)
        if args.mesh is None:
            parser.error("Plotting requires --mesh")
        
        mesh_file = open(args.mesh)
        vertices, triangles = read_off(mesh_file)

        
        x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]

        mesh = mlab.triangular_mesh(x, y, z, triangles, 
                                            representation='wireframe', opacity=0) 
        # mesh.mlab_source.dataset.cell_data.scalars = f 
        # mesh.mlab_source.dataset.cell_data.scalars.name = 'Cell data' 
        # mesh.mlab_source.update() 
        # mesh.parent.update()

        # mesh2 = mlab.pipeline.set_active_attribute(mesh, 
        #                 cell_scalars='Cell data') 

        mesh.mlab_source.dataset.point_data.scalars = f 
        mesh.mlab_source.dataset.point_data.scalars.name = 'Cell data' 
        mesh.mlab_source.update() 
        mesh.parent.update()

        mesh2 = mlab.pipeline.set_active_attribute(mesh, 
                        point_scalars='Cell data') 
        s2 = mlab.pipeline.surface(mesh2) 


        mlab.show()
        
        

In [97]:
f = open("torus.off")
vertices = np.loadtxt("2wcV.txt", dtype=float)
triangles = np.loadtxt("2wcT.txt", dtype=int) - 1
# vertices = np.load("vertices.npy")
# triangles = np.load("triangles.npy")
vertices, triangles = read_off(f)

In [90]:
cot_matrix, scaling_factor = generate_cot_matrix(vertices, triangles)

In [68]:
A_mixed = calc_A_mixed(vertices, cot_matrix, scaling_factor)

In [95]:
calc_A_mixed2(vertices, triangles, cot_matrix)

array([1.94937459e-01, 1.50862998e-01, 6.79315809e-07, 6.79315809e-07,
       1.50862998e-01, 1.94966694e-01, 1.50874908e-01, 6.79891536e-07,
       6.79840208e-07, 1.50874926e-01, 1.94974816e-01, 1.50851653e-01,
       6.79469419e-07, 6.79336715e-07, 1.50860248e-01, 1.94974816e-01,
       1.50860248e-01, 6.79349696e-07, 6.79469419e-07, 1.50851653e-01,
       1.94966694e-01, 1.50874926e-01, 6.79840208e-07, 6.79891536e-07,
       1.50874965e-01, 1.94937115e-01, 1.50862998e-01, 6.79315809e-07,
       6.79315809e-07, 1.50862998e-01, 1.94966694e-01, 1.50874908e-01,
       6.79891536e-07, 6.79840208e-07, 1.50874926e-01, 1.94974816e-01,
       1.50851653e-01, 6.79469419e-07, 6.79336715e-07, 1.50860248e-01,
       1.94974816e-01, 1.50860248e-01, 6.79349696e-07, 6.79469419e-07,
       1.50851653e-01, 1.94966694e-01, 1.50874926e-01, 6.79840208e-07,
       6.79891536e-07, 1.50874965e-01, 1.85402589e-01, 1.85135342e-01,
       1.96070838e-01, 1.85118549e-01, 1.51098184e-01, 6.12317154e-02,
      

In [102]:
A_mixed = calc_A_mixed3(vertices, triangles)

In [71]:
K_H = get_mean_curvature(vertices, triangles, A_mixed, cot_matrix)

In [72]:
K_G = get_gaussian_curvature(vertices, triangles, A_mixed)

In [73]:
K_1, K_2 = get_principal_curvatures(K_H, K_G)

True

In [20]:
x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]

mesh = mlab.triangular_mesh(x, y, z, triangles, 
                                    representation='wireframe', opacity=0) 
# mesh.mlab_source.dataset.cell_data.scalars = f 
# mesh.mlab_source.dataset.cell_data.scalars.name = 'Cell data' 
# mesh.mlab_source.update() 
# mesh.parent.update()

# mesh2 = mlab.pipeline.set_active_attribute(mesh, 
#                 cell_scalars='Cell data') 

mesh.mlab_source.dataset.point_data.scalars = f 
mesh.mlab_source.dataset.point_data.scalars.name = 'Cell data' 
mesh.mlab_source.update() 
mesh.parent.update()

mesh2 = mlab.pipeline.set_active_attribute(mesh, 
                point_scalars='Cell data') 
s2 = mlab.pipeline.surface(mesh2) 


mlab.show()

In [ ]:
A_mixed

In [ ]:
cot_matrix.shape

In [ ]:
diff = get_diff(vertices)

In [ ]:
cot_matrix * diff

In [ ]:
(np.array([[1, 1], [2, 2], [3, 3]]).reshape(-1, 1) - np.array([[1, 1], [2, 2], [3, 3]]).reshape(1, -1)).reshape(3, 3, 2)

In [ ]:
vertices.shape

In [ ]:
np.array([[1, 2], [2, 3], [3, 4]]).reshape(1, -1, 2) 

In [ ]:
np.linalg.norm((np.array([[1, 2], [2, 3], [3, 4]]).reshape(-1, 1, 2) - np.array([[1, 2], [2, 3], [3, 4]])), 2, axis=2)**2

In [ ]:
(np.array([[1, 2], [2, 3], [3, 4]]).reshape(-1, 1, 2) - np.array([[1, 2], [2, 3], [3, 4]]))

In [ ]:
np.multiply(np.array([[2, 2, 2], [3, 3, 3], [4, 4, 4]]), np.array([[[1, 2, 3], [4, 5, 6], [5, 6, 7]],
            [[1, 2, 3], [4, 5, 6], [5, 6, 7]],
         [[1, 2, 3], [4, 5, 6], [5, 6, 7]]]))

In [ ]:
np.array([[[1, 2, 3], [4, 5, 6], [5, 6, 7]],
            [[1, 2, 3], [4, 5, 6], [5, 6, 7]],
         [[1, 2, 3], [4, 5, 6], [5, 6, 7]]])

In [ ]:
a = np.random.randint(0, 10, (10, 3)) 

In [ ]:
a

In [ ]:
(np.arange(10) * a.T).T

In [ ]:
i = 0
triangles[(triangles[:, 0] == i) | (triangles[:, 1] == i) | (triangles[:, 2] == i)]

In [26]:
K_G_moebius = np.load("./surf_meshes/moebius_224/surface_KG.npy")

NameError: name 'K_G_morbius' is not defined

In [15]:
print(K_G_mother.tolist())

[-0.0004876675193627879, -0.005642662274633172, 0.003458857213300934, -0.006410520801866355, 0.0002635040698076345, -9.363936976361418e-06, 9.575289494374568e-05, 5.279910630160082e-05, -0.006647415562260618, 0.006136519301894243, -0.0051430072656504944, 9.402146411180582e-05, -0.003322021030944616, -0.009170937837287384, 0.002294760450714604, -0.0034285439334258137, 0.04459732345195736, 0.0009702403559404266, -0.0003899690303680153, 9.897664920244676e-05, 0.0031405170534520525, -0.0009330038232344746, -2.8450699230888384e-05, -0.008781634634679215, -0.00031219856564132997, -0.0003769257452770784, -0.006095152238661902, 0.025450751043273104, 1.735574800519963e-05, -0.0004511174264250213, 0.0004861229389060576, 0.00024061546481737286, 2.4746572160690805e-05, -0.0008743608041310092, -0.0004791568412331999, 0.0005331926891069895, 0.0023782682799265977, -0.005357652611035496, 0.003828807587555853, 0.00018927735966208417, -0.0005994474382223383, -0.007507848584639421, -0.003098421472588924,

In [28]:
K_1_moebius = np.load("surf_meshes/moebius_224/surface_K1.npy")

In [17]:
K_1_mother.tolist()

[0.013608089053642487,
 0.012534766726754963,
 0.06323180893148021,
 0.07918047556461641,
 0.02554297408055081,
 0.0021260267508445506,
 0.012564324396790482,
 0.010844810464814331,
 0.008033124778928064,
 0.08912479087042612,
 0.011388693330835099,
 0.03465223079639634,
 0.11743049255233931,
 0.024810718427030476,
 0.07363978607605332,
 0.03176271037274769,
 0.2541132529199166,
 0.03789874887525088,
 0.03239937634861343,
 0.012990669985942366,
 0.0633208718303632,
 0.013274990232572127,
 0.005257858637765174,
 0.05979432792318949,
 0.01273501678563823,
 0.006028027518139175,
 0.049243226065093085,
 0.19099162943837633,
 0.006059059992431349,
 0.037688391048016315,
 0.12582104853163054,
 0.016763683187474347,
 0.006708512060113139,
 0.10515698506725478,
 0.007440694969305177,
 0.06304191879115446,
 0.07680436566387316,
 0.08440975540621268,
 0.061637403804879476,
 0.020465132554160236,
 0.06723557866412327,
 0.03739053552302868,
 0.036148995617233504,
 0.11447142565022439,
 0.046817293

In [30]:
K_2_moebius = np.load("surf_meshes/moebius_224/surface_K2.npy")
K_2_moebius.tolist()

[0.24254103067632443,
 -0.00812924916953419,
 0.14778018745813765,
 -0.012687534827758725,
 0.24221681521702462,
 -0.010812430271138332,
 0.24139672650496158,
 -0.009284468671011022,
 0.2401045859736905,
 -0.008111201087097428,
 0.14659934775269542,
 -0.006848053185365377,
 0.14581427016132192,
 -0.006297422177324034,
 0.14488732380545555,
 -0.005905552637292609,
 0.23143208823976552,
 -0.005854961568874703,
 0.2287482530544541,
 -0.005680632002379185,
 0.22599796394644422,
 -0.005620795261523752,
 0.22325866034305608,
 -0.005663451923069461,
 0.1389570852497779,
 -0.005647919823742367,
 0.1376578101831992,
 -0.0058829794972894145,
 0.13635990393377953,
 -0.006214226631140487,
 0.13507189827229132,
 -0.006647465184379666,
 0.13280405955530905,
 -0.007203400319510403,
 0.14952224555253618,
 -0.008434211016299605,
 0.18853200948231988,
 -0.009385658015716725,
 0.1863184650613427,
 -0.010540031451094133,
 0.184076096675515,
 -0.01194960170101135,
 0.1818284754454518,
 -0.01361588331212307

In [31]:
np.linalg.norm((K_1_moebius * K_2_moebius) - K_G_moebius, ord=2)

1.080058007149702

In [32]:
K_H_moebius = np.load("surf_meshes/moebius_224/surface_KH.npy")

In [33]:
K_H_moebius

array([2.42541031e-01, 1.49452256e-03, 1.47780187e-01, 7.89988223e-05,
       2.42216815e-01, 2.16820674e-03, 2.41396727e-01, 4.30000107e-03,
       2.40104586e-01, 6.41143357e-03, 1.46599348e-01, 6.75210633e-03,
       1.45814270e-01, 8.37344122e-03, 1.44887324e-01, 9.95136073e-03,
       2.31432088e-01, 1.44193183e-02, 2.28748253e-01, 1.62505602e-02,
       2.25997964e-01, 1.79732635e-02, 2.23258660e-01, 1.95730630e-02,
       1.38957085e-01, 1.67039534e-02, 1.37657810e-01, 1.77124705e-02,
       1.36359904e-01, 1.85620598e-02, 1.35071898e-01, 1.90691382e-02,
       1.32804060e-01, 1.94370565e-02, 1.49522246e-01, 3.39535216e-02,
       1.88532009e-01, 3.36835767e-02, 1.86318465e-01, 3.29510757e-02,
       1.84076097e-01, 3.17053325e-02, 1.81828475e-01, 2.99788770e-02,
       1.79625503e-01, 2.77273020e-02, 1.77502508e-01, 1.91585822e-02,
       2.25259707e-02, 1.18967041e-02, 1.14709600e-02, 9.95070524e-03,
       1.43843258e-01, 1.44887589e-01, 8.37329776e-03, 1.45814713e-01,
      

In [34]:
K_G_moebius

array([ 1.59066056e-01, -9.03833846e-05,  9.72233163e-02, -1.62978141e-04,
        1.59093165e-01, -1.63795817e-04,  1.59276944e-01, -1.66047809e-04,
        1.59628211e-01, -1.69800437e-04,  9.85069508e-02, -1.39373399e-04,
        9.93346921e-02, -1.45119715e-04,  1.00384000e-01, -1.52412121e-04,
        1.63594153e-01, -2.03129684e-04,  1.65502036e-01, -2.16896485e-04,
        1.67903783e-01, -2.33641408e-04,  1.70873241e-01, -2.53776890e-04,
        1.09912835e-01, -2.20584178e-04,  1.12885547e-01, -2.43013650e-04,
        1.16280837e-01, -2.69314305e-04,  1.20115023e-01, -2.97711659e-04,
        1.23475480e-01, -3.31914774e-04,  1.45671117e-01, -6.43878248e-04,
        1.92798639e-01, -7.20375639e-04,  2.00221728e-01, -8.05703011e-04,
        2.07951049e-01, -9.00525170e-04,  2.15817884e-01, -1.00177006e-03,
        2.23660882e-01, -1.10890216e-03,  2.31217135e-01, -9.34648720e-04,
       -5.33048500e-04, -3.21245314e-04, -1.61472340e-04, -1.52414385e-04,
        1.01679534e-01,  

In [63]:
np.load("torus_KH.npy")

array([0.95023572, 0.90978988, 1.25887732, 1.25887732, 0.90978988,
       0.94993456, 0.90987046, 1.25896677, 1.25908385, 0.90984754,
       0.9498125 , 0.91008216, 1.25765421, 1.25771751, 0.91008406,
       0.9498125 , 0.91008406, 1.25770282, 1.25765421, 0.91008216,
       0.94993456, 0.90984754, 1.25908385, 1.25896677, 0.90986965,
       0.95002607, 0.90978988, 1.25887732, 1.25887732, 0.90978988,
       0.94993456, 0.90987046, 1.25896677, 1.25908385, 0.90984754,
       0.9498125 , 0.91008216, 1.25765421, 1.25771751, 0.91008406,
       0.9498125 , 0.91008406, 1.25770282, 1.25765421, 0.91008216,
       0.94993456, 0.90984754, 1.25908385, 1.25896677, 0.90986965,
       0.7473288 , 0.73839023, 0.88504708, 0.73868506, 0.89927739,
       0.78780903, 0.76926544, 0.76905812, 1.04475284, 0.38185542,
       0.24809566, 0.24658847, 1.04447171, 0.78786641, 0.76928844,
       0.76905812, 0.89928553, 0.74712322, 0.73851126, 0.73868346,
       0.7471005 , 0.73864955, 0.88504828, 0.8994337 , 0.78797

NameError: name 'K_1' is not defined

In [87]:
np.where(triangles == np.array([227, 201, 121]))

TypeError: where() takes no keyword arguments

In [86]:
triangles

array([[227, 201, 151],
       [227, 201, 107],
       [253,  90, 120],
       ...,
       [784, 726, 157],
       [784, 189, 157],
       [784, 718, 189]])

In [88]:
vertices

array([[-0.20129938,  0.91170195, -0.35815934],
       [-0.339349  ,  0.85309019, -0.39633242],
       [-0.32435836,  0.9059065 , -0.27225919],
       ...,
       [-0.19286529, -0.65620908,  0.72951534],
       [ 0.44999316, -0.4044365 ,  0.79620178],
       [ 0.23530679, -0.16918055,  0.95708341]])